In [ ]:
from keras.src.saving import load_model

model = load_model("data/ckpt/best.keras")

In [ ]:
from src.data_loader import load_dataframe, create_datasets, denormalize_labels

df = load_dataframe("data/cleaned_metadata.csv")

train_dataset, test_dataset = create_datasets(
    df,
    test_size=0.2,
    batch_size=128,
    augment_fn=None
)

In [ ]:
%env TF_USE_LEGACY_KERAS=0

In [ ]:
import tensorflow as tf

def representative_data_gen():
    for data, _ in test_dataset.take(100):
        yield [data]


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_int8_model = converter.convert()

output_path = "data/ckpt/best_int8.tflite"
with open(output_path, "wb") as f:
    f.write(tflite_int8_model)

print(f"Model saved at: {output_path}")

In [ ]:
import tensorflow as tf

output_path = "data/ckpt/best.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open(output_path, "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite and saved as 'best.tflite'")

# Test Inference API

In [ ]:
import numpy as np
from PIL import Image
import requests

URL = "http://192.168.31.242:5000"

def send_image(image_path):
    with open(image_path, 'rb') as img_file:
        files = {'image': img_file}
        response = requests.post(f"{URL}/predict", files=files)

    if response.status_code == 200:
        result = response.json()
        print("Prediction:", result['prediction'])
        print("Inference Time (seconds):", result['inference_time_seconds'])
        return result['prediction']
    else:
        print("Error:", response.json())

for (Xs, ys) in test_dataset.take(1):
    X, y = Xs[0] * 255, ys[0]
    print(denormalize_labels(y))
    print(X.shape)
    image_path = "test_image.jpg"
    big_model_pred = model.predict(Xs)[0]
    grayscale_array_squeezed = np.squeeze(X, axis=2)
    print(grayscale_array_squeezed.shape)
    Image.fromarray(grayscale_array_squeezed.astype(np.uint8), mode="L").save(image_path)
    print("Quantised predictions", denormalize_labels(send_image('test_image.jpg')))
    print("Best model predictions", denormalize_labels(big_model_pred))
    print("Real values", denormalize_labels(y).numpy())


In [ ]:
import numpy as np
from PIL import Image
import requests
from tqdm import tqdm
import time
from sklearn.metrics import mean_squared_error

URL = "http://192.168.31.242:5000"

def send_image(image_path):
    with open(image_path, 'rb') as img_file:
        files = {'image': img_file}
        response = requests.post(f"{URL}/predict", files=files)

    if response.status_code == 200:
        result = response.json()
        return result['prediction'], result['inference_time_seconds']
    else:
        print("Error:", response.json())
        return None, None

def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, rmse

big_model_preds = []
quantized_preds = []
real_values = []
inference_times = []

for Xs, ys in tqdm(test_dataset, desc="Processing test samples"):
    for X, y in zip(Xs, ys):
        X, y = X * 255, y
        real_values.append(denormalize_labels(y).numpy())

        image_path = "test_image.jpg"
        grayscale_array_squeezed = np.squeeze(X, axis=2)
        Image.fromarray(grayscale_array_squeezed.astype(np.uint8), mode="L").save(image_path)

        big_model_pred = model.predict(np.expand_dims(X / 255, axis=0))[0]
        big_model_preds.append(denormalize_labels(big_model_pred))

        quant_pred, inference_time = send_image(image_path)
        quantized_preds.append(denormalize_labels(np.array(quant_pred)))
        inference_times.append(inference_time)

In [ ]:
print(f'Big model predictions: {np.array(big_model_preds).shape}')
print(f'Quantized model predictions: {np.array(quantized_preds).shape}')
quantized_preds = np.reshape(np.array(quantized_preds), (-1, 3))
mse_big, rmse_big = calculate_metrics(real_values, big_model_preds)
mse_quant, rmse_quant = calculate_metrics(real_values, quantized_preds)
avg_inference_time = np.mean(inference_times)

print("\nComparison Metrics:")
print(f"Big Model - MSE: {mse_big:.4f}, RMSE: {rmse_big:.4f}")
print(f"Quantized Model - MSE: {mse_quant:.4f}, RMSE: {rmse_quant:.4f}")
print(f"Average Inference Time (Quantized Model): {avg_inference_time:.4f} seconds")